In [ ]:
'''
In this notebook, providing information about how to use Librosa and Pydub library for.
A. Using Librosa (librosa.effects.split), Get the non-silent intervals in given audio file
B. Using Pydub (split_on_silence), split the audio file into multiple chunks with non-silent data. Means splitted data wont contain any silent data
C. Using Pydub (detect_nonsilent), get the non-silent intervals in given audio file 
D. Using Pydub (detect_silence), get the silent intervals in given audio file 
E. Using FFMPEG, get the non-silent intervals in given audio file

'''

In [ ]:
!pip install pydub

In [2]:
from __future__ import print_function

import datetime
import copy
import numpy as np
import pysrt
import time
import subprocess
from os import path
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

import soundfile as sf

import librosa
import librosa.display as display

from pydub import AudioSegment
from pydub.silence import  detect_nonsilent
from pydub.silence import split_on_silence
from pydub.silence import detect_silence

In [ ]:
def detect_silence_using_ffmpeg():
    '''
    Command line command to detect silence range using ffmpeg
    ffmpeg -i "input.mov" -af silencedetect=noise=-30dB:d=0.5 -f null - 2> vol.txt
    '''
    input_video_file = 'D:/Abhishek/Machine Learning Models/Subtitle-Sync/github-repo/SubtitleSynchronizer/data/THENUN.mp4'

    extract_sound_cmd = [
         'ffmpeg',
         '-i', input_video_file,
         '-af', 'silencedetect=noise=-30dB:d=0.5',
         '-f', 'null', '-', '2>',
         'vol.txt'
     ]
    output = subprocess.call(extract_sound_cmd)
    print(output)

In [173]:
input_audio_file = 'D:/Abhishek/Machine Learning Models/Subtitle-Sync/video/New/THENUN.flac'
samples, sr = librosa.load(input_audio_file)
print('duration: ', len(samples)/sr)

non_silent_interval = librosa.effects.split(samples)  #top_db -> this parameter can also be specified.

for silence_range in non_silent_interval:
    start_second = silence_range[0]/sr
    end_second = silence_range[1]/sr
    
    start = str(datetime.timedelta(seconds=start_second))
    end = str(datetime.timedelta(seconds=end_second))
    
    print(start, ' : ', end)

[0. 0. 0. ... 0. 0. 0.]


In [187]:
filename = 'D:/Abhishek/Machine Learning Models/Subtitle-Sync/video/New/output/THENUN/vocals.wav'
audio_signal = AudioSegment.from_file(filename, "wav")

channel_count = audio_signal.channels    #Get channels
sample_width = audio_signal.sample_width #Get sample width
duration_in_sec = len(audio_signal) / 1000#Length of audio in sec
sample_rate = audio_signal.frame_rate

print ("sample_width=", sample_width )
print ("channel_count=", channel_count)
print ("duration_in_sec=", duration_in_sec) 
print ("frame_rate=", sample_rate)
print('lenght :', len(audio_signal))

sample_width= 2
channel_count= 2
duration_in_sec= 600.0
frame_rate= 44100
lenght : 600000


In [ ]:
chunks = split_on_silence(audio_signal,min_silence_len=280,silence_thresh=-33,keep_silence=150)
for i, chunk in enumerate(chunks):
    print(i, ' : ', len(chunk))
    print("\n")

In [ ]:
audio_signal = chunks[0]
channel_count = audio_signal.channels    #Get channels
sample_width = audio_signal.sample_width #Get sample width
duration_in_sec = len(audio_signal) / 1000#Length of audio in sec
sample_rate = audio_signal.frame_rate

print ("sample_width=", sample_width )
print ("channel_count=", channel_count)
print ("duration_in_sec=", duration_in_sec) 
print ("frame_rate=", sample_rate)
print('lenght :', len(audio_signal))

audio_signal.export("pydub_mute_wave.wav", format="wav")

In [ ]:
sound = AudioSegment.from_file("D:/Abhishek/Machine Learning Models/Subtitle-Sync/vocal_mute_signals/vocals/video000/vocals.wav", format="mp3")

silent_ranges = detect_silence(sound, min_silence_len=1000, silence_thresh=-33)
for silence_range in silent_ranges:
    start = str(datetime.timedelta(seconds=silence_range[0]/1000))
    end = str(datetime.timedelta(seconds=silence_range[1]/1000))
    print(start, ' : ', end)
    
    
nonsilent_audio_rnage = detect_nonsilent(sound,min_silence_len=1000,silence_thresh=-33)
for silence_range in nonsilent_audio_rnage:
    start = str(datetime.timedelta(seconds=silence_range[0]/1000))
    end = str(datetime.timedelta(seconds=silence_range[1]/1000))
    print(start, ' : ', end)